___
# Importing necessary libraries
___

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')


import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px


import cufflinks as cf
cf.go_offline()

In [19]:
# Loading the dataset
df = pd.read_csv('../dataset/COVID-19_Sentiments.csv')

In [20]:
df.head()

,Text_Id,Text,Date,Location,Sentiments
0,1241032866567356417,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,"Uttar Pradesh, India",0.000
1,1241032867699765249,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:27 +0000 2020,Mumbai,0.000
2,1241032875102703616,"RT @SmokingSkills_: Daughter of an IAS, son of...",Fri Mar 20 16:04:29 +0000 2020,"Jodhpur, India",0.350
3,1241032877099237379,RT @narendramodi: The young actors have someth...,Fri Mar 20 16:04:29 +0000 2020,"Gurugram, Bharat",0.125
4,1241032870405128192,RT @theskindoctor13: Shaheen Bagh is still on....,Fri Mar 20 16:04:28 +0000 2020,"New Delhi, India",0.000


In [21]:
df.drop('Text_Id' , axis = 1 , inplace = True)

In [22]:
 # Retaining only date in Date feature
df['Date'] = pd.to_datetime( df['Date'] ).dt.date

In [23]:
df.head()

,Text,Date,Location,Sentiments
0,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000
1,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000
2,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350
3,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125
4,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Text             648958 non-null  object 
 1   Date             648958 non-null  object 
 2   Location         648958 non-null  object 
 3   Sentiments       637978 non-null  float64
 4   Sentiment Score  648958 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 24.8+ MB


In [24]:
lis = []
for value in df.Sentiments:
    if value < 0 :
        lis.append(-1)
    elif value == 0 :
        lis.append(0)
    else:
        lis.append(1)
df['Sentiment Score'] = lis 
df.head()

,Text,Date,Location,Sentiments,Sentiment Score
0,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [25]:
# Getting data of only first phase of lockdown
import datetime
phase_first = df[ (df.Date >= datetime.date(2020, 3, 25)) & (df.Date <= datetime.date(2020, 4, 14)) ]

In [26]:
phase_first

,Text,Date,Location,Sentiments,Sentiment Score
23828,"RT @chathootti: Even this national emergency, ...",2020-03-25,"Tamil Nadu, India",NaN,1
23829,Now the slogan has been justified................,2020-03-25,India,NaN,1
23830,RT @Shehzad_Ind: Thank you @narendramodi @Amit...,2020-03-25,"Hindustan, India",NaN,1
23831,RT @jvin_tootu: Idea in the time of Corona\n A...,2020-03-25,"Aluva, India",NaN,1
23832,RT @RajiniFollowers: Super Bro 🙌🙌🙌🙌 #RAJINIVoi...,2020-03-25,"Bengaluru South, India",NaN,1
...,...,...,...,...,...
491428,RT @UnderpaidBanke8: No Insurance cover\nNo Re...,2020-04-14,"Haryana, India",-0.178571,-1
491429,RT @iAnkurSingh: 21 days would've been suffici...,2020-04-14,India,-0.071429,-1
491430,RT @Sanjaythech1234: No Insurance cover\nNo Re...,2020-04-14,"Haryana, India",-0.178571,-1
491431,RT @IYC_UPWest: There can not be a better day ...,2020-04-14,"New Delhi, India",0.500000,1


In [27]:
# Counting tweets only for +ve , -ve and neutral tweets
positive = list(phase_first['Sentiment Score']).count( 1 )
negative = list(phase_first['Sentiment Score']).count(-1 )
neutral  = list(phase_first['Sentiment Score']).count( 0 )

lis = [['positive' , positive] , ['negative' , negative] , ['neutral' , neutral]]
distribution = pd.DataFrame(lis , columns = ['Category' , 'Count'])
distribution

,Category,Count
0,positive,115308
1,negative,54504
2,neutral,116440


In [28]:
px.pie( distribution , values='Count', names='Category', title='First phase Tweets Category distribution' ,color_discrete_sequence=px.colors.sequential.RdBu)

In [29]:
df.head()

,Text,Date,Location,Sentiments,Sentiment Score
0,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"Uttar Pradesh, India",0.000,0
1,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,Mumbai,0.000,0
2,"RT @SmokingSkills_: Daughter of an IAS, son of...",2020-03-20,"Jodhpur, India",0.350,1
3,RT @narendramodi: The young actors have someth...,2020-03-20,"Gurugram, Bharat",0.125,1
4,RT @theskindoctor13: Shaheen Bagh is still on....,2020-03-20,"New Delhi, India",0.000,0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Text             648958 non-null  object 
 1   Date             648958 non-null  object 
 2   Location         648958 non-null  object 
 3   Sentiments       637978 non-null  float64
 4   Sentiment Score  648958 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 24.8+ MB


In [30]:
df1 = df.drop(['Text', 'Sentiments', 'Location'], axis = 1)

In [31]:
df1.head()

,Date,Sentiment Score
0,2020-03-20,0
1,2020-03-20,0
2,2020-03-20,1
3,2020-03-20,1
4,2020-03-20,0


In [36]:
df1['Date'] = pd.to_datetime( df1['Date'] )

In [37]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648958 entries, 0 to 648957
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             648958 non-null  datetime64[ns]
 1   Sentiment Score  648958 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 9.9 MB


In [38]:
df1['month']  = df1['Date'].dt.month
df1['day'] = df1['Date'].dt.day

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [40]:
x = df1[['month', 'day']]
y = df1['Sentiment Score']
classifier = LogisticRegression()
classifier.fit(x, y)
classifier.score(x,y)

0.40418023970734623